# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [38]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn
from azureml.automl.runtime.onnx_convert import OnnxConverter
import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [39]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Heart-Disease-Predection'

experiment=Experiment(ws, experiment_name)

In [40]:
#Creating Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
#Create compute cluster


# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)
    print('cpu-cluster has been created')


Found existing compute target


In [41]:
ds = Dataset.get_by_name(ws, name='Heart Failure Dataset')
data = ds.to_pandas_dataframe()
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


In [42]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(data, test_size=0.20)

In [43]:
from azureml.data.dataset_factory import TabularDatasetFactory
pd.DataFrame(training_data).to_csv("data/train_data.csv")
df=ws.get_default_datastore()
df.upload(src_dir='./data', target_path='HeartDataset', overwrite=True,show_progress=True)
train_data=TabularDatasetFactory.from_delimited_files(path=df.path('HeartDataset/train_data.csv'))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [44]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 25,
                    "max_concurrent_iterations": 4,
                    "n_cross_validations": 5,
                    "primary_metric": 'accuracy',
                        
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(  
    task='classification',
    compute_target=cpu_cluster,
    training_data= train_data,
    label_column_name ='DEATH_EVENT',
    enable_onnx_compatible_models=True,
    **automl_settings
)

In [45]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_c6807fca-e917-45e6-a9ac-e979181109dc

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, 

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [46]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [47]:
best_automl_run,fitted_model = remote_run.get_output()

In [48]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Heart-Disease-Predection,AutoML_c6807fca-e917-45e6-a9ac-e979181109dc_73,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [49]:
best_automl_run.get_metrics()

{'weighted_accuracy': 0.9192119514614235,
 'norm_macro_recall': 0.6589471207687526,
 'recall_score_macro': 0.8294735603843764,
 'precision_score_macro': 0.8812817362817362,
 'matthews_correlation': 0.7080947838665682,
 'precision_score_micro': 0.8828014184397162,
 'log_loss': 0.3520949108342314,
 'average_precision_score_macro': 0.8821121900700654,
 'f1_score_micro': 0.8828014184397162,
 'recall_score_weighted': 0.8828014184397162,
 'AUC_micro': 0.9252558189980384,
 'recall_score_micro': 0.8828014184397162,
 'AUC_weighted': 0.9140100754189937,
 'average_precision_score_micro': 0.9274616828310499,
 'average_precision_score_weighted': 0.914738031353739,
 'precision_score_weighted': 0.8828862227798397,
 'accuracy': 0.8828014184397162,
 'f1_score_weighted': 0.8782156507736539,
 'f1_score_macro': 0.8483255012531329,
 'balanced_accuracy': 0.8294735603843764,
 'AUC_macro': 0.9140100754189939,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_c6807fca-e917-45e6-a9ac-e979181109dc

In [50]:
#TODO: Save the best model
best_automl_run,onnx_model =remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model,file_path="./automl_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [58]:
#Register the best automl model,
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [61]:
script_file_name ='inference/sscore.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py','inference/sscore.py')

In [62]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_automl_run.get_environment())

In [64]:

#Local Deployment
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration(port=9001)
local_service = Model.deploy(ws, "test", [automl_model], inference_config, local_config)
local_service.wait_for_deployment(show_output=True)

#Set up the deployment_config as webservice
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

#Deploy the model
service = Model.deploy(
    workspace = ws,
    name = "heart-failure-model",
    models = [automl_model],
    inference_config = inference_config,
    deployment_config = aci_config, overwrite=True)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a
 ---> c354c32ff2d1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 459a0323787f
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjUxMGI5NGJhLWU0NTMtNDQxNy05ODhiLWZiZGMzN2I1NWNhNyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNDA5NSIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNDA5NSIsIndvcmtzcGFjZUlkIjoiY2QyZTRhNDQtYTY3Mi00YWYzLWFlMWItZmFhMzZmMmRjMzQwIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 6b24096fc115
 ---> 760bbf76c6da
Step 4/5 : RUN mv '/var/azureml-app/tmp9yp6gqgo.py' /var/azureml-app/main.py
 ---> Running in 86a1d697d333
 ---> 6d49a68afd7f
Step 5/5 : 

In [66]:
model_uri = service.scoring_uri[]

SyntaxError: invalid syntax (<ipython-input-66-45f3785f4d78>, line 1)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [69]:
scoring_uri='http://136e0723-f0f3-4005-b8cb-5ee23554837e.southcentralus.azurecontainer.io/score'
key=''
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'
data = {"data":
        [
          {
            "age": 52,
            "sex": 1,
            "cp": 3,
            "trestbps": 145,
            "chol": 233,
            "fbs": 1,
            "restecg": 0,
            "thalach": 150,
            "exang": 0,
            "oldpeak": 2.3,
            "slope": 0,
            "ca": 0,
            "thal": 1,
          }
      ]
    }
test_sample = json.dumps(data)


resp = requests.post(scoring_uri, test_sample, headers=headers)
print(resp.json())


{"error": "DataException:\n\tMessage: Expected column(s) cp not found in fitted data.\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Expected column(s) cp not found in fitted data.\",\n        \"target\": \"X\",\n        \"inner_error\": {\n            \"code\": \"BadArgument\",\n            \"inner_error\": {\n                \"code\": \"MissingColumnsInData\"\n            }\n        },\n        \"reference_code\": \"17049f70-3bbe-4060-a63f-f06590e784e5\"\n    }\n}"}


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
#Print the webservice logs
print(service.get_logs())

In [ ]:
#Delete the service
service.delete()